# 从网页中截取文章内容

## 步骤1：参数设置

In [11]:
urls = [
    'http://www.cnblogs.com/qw12/p/6294430.html'
    ,'http://www.cnblogs.com/neopenx/p/4453161.html'
]
url = urls[1]
print(url)

http://www.cnblogs.com/neopenx/p/4453161.html


## 步骤2：查找相应的截取关键字

In [12]:
if 'blog.jobbole.com' in url:  # 伯乐在线
    title_key = '.entry-header'
    content_key = '.entry'
    print('伯乐在线 http://blog.jobbole.com')
elif 'blog.csdn.net' in url:  # csdn
    title_key = '.link_title'
    content_key = '#article_content'
    print('csdn http://blog.csdn.net')
elif 'www.codingpy.com' in url:  # 编程派网址
    title_key = '.header h1'
    content_key = '.article-content'
    print('编程派网址 http://www.codingpy.com')
elif 'www.cnblogs.com' in url:  # 博客园
    title_key = '#cb_post_title_url'
    content_key = '#cnblogs_post_body'
    print('博客园 http://www.cnblogs.com')
else:
    title_key = '#keraspython'
    content_key = '.section'
    print('其它')

博客园 http://www.cnblogs.com


## 步骤3：发出网页请求，接收响应

In [13]:
from urllib import request
req = request.Request(url)
res = request.urlopen(req)
html = res.read().decode('utf-8')
print(html)


<!DOCTYPE html>
<html lang="zh-cn">
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width, initial-scale=1" />
<title>ReLu(Rectified Linear Units)激活函数 - Physcal - 博客园</title>
<link type="text/css" rel="stylesheet" href="/bundles/blog-common.css?v=ChDk9h03-S75WEqNhGvXkWireJ5cCWdK1xRM9NIXfnM1"/>
<link id="MainCss" type="text/css" rel="stylesheet" href="/skins/iMetro/bundle-iMetro.css?v=2TeLnpOZe7KA__aXpZGlfXgoc-73meGsojRtEQ8QCJA1"/>
<link type="text/css" rel="stylesheet" href="/blog/customcss/199930.css?v=0GV7QRXOw1y0HIAInolzRwfOQy0%3d"/>
<link id="mobile-style" media="only screen and (max-width: 768px)" type="text/css" rel="stylesheet" href="/skins/iMetro/bundle-iMetro-mobile.css?v=GdPsEX6D7age8xvCjF--7m7O4Z1PVL_D4zGOS2mXsDg1"/>
<link title="RSS" type="application/rss+xml" rel="alternate" href="http://www.cnblogs.com/neopenx/rss"/>
<link title="RSD" type="application/rsd+xml" rel="EditURI" href="http://www.cnblogs.com/neopenx/rsd.xml"/>
<link type="applicatio

## 步骤4：从网页中提取文章标题和内容

In [14]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
title = soup.select(title_key)[0].text.strip()  # 文章标题
print('文章标题：', title)
content = soup.select(content_key)[0]  # 文章内容

文章标题： ReLu(Rectified Linear Units)激活函数


## 步骤5：下载文章中的图片

### 5.1 哈希码生成函数，用于给图片重新命名

In [15]:
import hashlib
def md5(name):
    """ 将字符串转成哈希码 """
    if not isinstance(name, str):
        name = str(name)
    md5 = hashlib.md5()
    md5.update(name.encode('utf-8'))
    return md5.hexdigest()

### 5.2 下载图片，并修改文章图片的超链接

In [16]:
import re
import os
content = str(content)
pattern = '<img .*?src=\"(.*?)\"'
re_image = re.compile(pattern)
for image_link in re_image.findall(content):
    if not os.path.exists('images'):
        os.mkdir('images')
    filename = 'images/' + md5(image_link) + os.path.splitext(image_link)[-1]
    try:
        request.urlretrieve(image_link, filename)
        print('下载完成', filename)
    except Exception as e:
        print('图片出错', e)
    else:
        content = content.replace(image_link, filename)
print('已完成--------')

下载完成 images/d6a14abc8b3b7b79dec5e77ae3f44026.png
图片出错 <urlopen error [Errno 2] No such file or directory: '/C:\\Users\\Administrator\\AppData\\Roaming\\Tencent\\Users\\2876734969\\QQ\\WinTemp\\RichOle\\5{`L6GE1KSSOWDQ`GUE3183.png'>
图片出错 <urlopen error [Errno 2] No such file or directory: '/C:\\Users\\Administrator\\AppData\\Roaming\\Tencent\\Users\\2876734969\\QQ\\WinTemp\\RichOle\\5{`L6GE1KSSOWDQ`GUE3183.png'>
下载完成 images/fceb82b99c19947b59c37cd0cc41c4de.png
下载完成 images/a3e1d58e1b709a03dcf55d784c0784ea.png
下载完成 images/2bb487cf0996227967d675cccbd398e7.png
下载完成 images/9c47fbf71b3d04638a9de2348fc23439.png
下载完成 images/bb9aae5c1ed830733990036685344cc9.png
下载完成 images/4f41f81fdba33863cda1688406035d3b.png
已完成--------


## 步骤6：将截取的文章标题和内容重新组合成新的网页文件

In [17]:
html_template = """<!DOCTYPE html>
<html><head><meta charset="UTF-8">
</head><body>
<p><a href="{origin}">原文链接</a></p>
<p><center><h1>{title}</h1></center></p>
    {content}
</body></html>"""
html = html_template.format(origin=url, title=title, content=content)
print(html)

<!DOCTYPE html>
<html><head><meta charset="UTF-8">
</head><body>
<p><a href="http://www.cnblogs.com/neopenx/p/4453161.html">原文链接</a></p>
<p><center><h1>ReLu(Rectified Linear Units)激活函数</h1></center></p>
    <div id="cnblogs_post_body"><h3>论文参考：<a href="http://wenku.baidu.com/link?url=GtIz68_egY1kBAFgt3LY0f2v89H6doReu6pcNB34yHiG_PEiSekhZVtSszGB28dtRmEfAKrYOQtu5T0EVLeTzKwpZAeTJEfo00XOgbj9l4u" target="_blank">Deep Sparse Rectifier Neural Networks</a> (很有趣的一篇paper）</h3>
<h2>起源：传统激活函数、脑神经元激活频率研究、稀疏激活性</h2>
<h3>传统Sigmoid系激活函数</h3>
<p><img alt="" height="329" src="images/d6a14abc8b3b7b79dec5e77ae3f44026.png" width="438"/></p>
<p>传统神经网络中最常用的两个激活函数，Sigmoid系（Logistic-Sigmoid、Tanh-Sigmoid）被视为神经网络的核心所在。</p>
<p>从数学上来看，非线性的Sigmoid函数对中央区的信号增益较大，对两侧区的信号增益小，在信号的特征空间映射上，有很好的效果。</p>
<p>从神经科学上来看，中央区酷似神经元的兴奋态，两侧区酷似神经元的抑制态，因而在神经网络学习方面，可以将重点特征推向中央区，将非重点特征推向两侧区。</p>
<p>无论是哪种解释，看起来都比早期的线性激活函数(y=x),阶跃激活函数(-1/1,0/1)高明了不少。</p>
<h3>近似生物神经激活函数：Softplus&amp;ReLu </h3>
<p>2001年，神经科学家Dayan、Abott从生物学角度，模拟出了脑神经元接受信号更精确的

## 步骤7：将文件写入磁盘

In [18]:
filename = title + ".html"
with open(filename, "w") as f:
    f.write(html.replace(u'\xa0', u' ').replace(u'\U0001f60a', u' '))  # 在windows中出错，所以这里进行了字符串替换

## (完)